In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import TextVectorization
import gradio as gr


In [2]:
df=pd.read_csv("jigsaw-toxic-comment-classification-challenge/train.csv")

In [3]:
X=df['comment_text']
y=df[df.columns[2:]].values

In [4]:
MAX_FEATURES=200000

In [5]:
vectorizer=TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')

In [6]:
vectorizer.adapt(X.values)

In [7]:
vectorized_text=vectorizer(X.values)

In [8]:
#MCSHBAP -map,cache,shuffle,batch,prefetch  from_tensor_slices, list_dir

dataset=tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset=dataset.cache()
dataset=dataset.shuffle(160000)
dataset=dataset.batch(16)
dataset=dataset.prefetch(8)  #helps prevents bottlenecks

In [9]:
batch_X,batch_y=dataset.as_numpy_iterator().next()

In [10]:
train=dataset.take(int(len(dataset)*.7))
val=dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*.2))
test=dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [11]:
model=tf.keras.models.load_model('Comment_Toxicity_with_epoch_3.h5')

In [12]:
def score_comment(comment):
    vectorized_comment=vectorizer([comment])
    results=model.predict(vectorized_comment)
    
    text=''
    for idx,col in enumerate(df.columns[2:]):
        text+='{}:{}\n'.format(col,results[0][idx]>0.5)
    
    return text

In [13]:
interface=gr.Interface(fn=score_comment,inputs=gr.inputs.Textbox(lines=2,placeholder='Comment to score'),
                      outputs='text',
                      description="Comment and know its toxicity...\n Created by Tushar Gahtori",
                      title="Comment Toxicity Analysis")

C:\Users\Tusha\AppData\Roaming\Python\Python39\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\Tusha\AppData\Roaming\Python\Python39\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\Tusha\AppData\Roaming\Python\Python39\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [14]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://6bed1173-4bc7-4041.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 1s 1s/step
